In [1]:
import pandas as pb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import os.path
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
import gzip

In [4]:
X_L = []
X_R = []
Y = []

for root, dirs, files in os.walk('Recording Data'):
    for file in files:
        if file.endswith(".txt") and file != 'Subject codes.txt':
            file_path = os.path.join(root, file)
            match root[-1]:
                case '1' | '2':
                    has_dyslexia = 1
                case _:
                    has_dyslexia = 0
            data = pd.read_csv(file_path, sep='\t', index_col=None, decimal=",")
            data['r_dist'] = (data['RX'].diff() ** 2 + data['RY'].diff() ** 2) ** 0.5
            data['l_dist'] = (data['LX'].diff() ** 2 + data['LY'].diff() ** 2) ** 0.5
            X_L.append(data['l_dist'][1:])
            X_R.append(data['r_dist'][1:])
            Y.append(has_dyslexia)

In [7]:
X = X_L + X_R
Y = Y * 2

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [9]:
k_m = [1, 5, 10, 15, 20, 25]

In [10]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

f_scores = []
for k in k_m:
    for train_index, test_index in kf.split(X):
        X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
        Y_train, Y_test = [Y[i] for i in train_index], [Y[i] for i in test_index]

        predicts = []
        for x1 in X_test:
            x1 = np.array(x1)
            Cx1 = len(gzip.compress(x1))
            distance_from_x1 = []
            for x2 in X_train:
                x2 = np.array(x2)
                Cx2 = len(gzip.compress(x2))
                x1x2 = np.concatenate((x1, x2))
                Cx1x2 = len(gzip.compress(np.array(x1x2)))
                ncd = (Cx1x2 - min(Cx1 , Cx2)) / max (Cx1 , Cx2 )
                distance_from_x1.append(ncd)
                        
            sorted_idx = np.argsort(np.array(distance_from_x1))
            top_k_class = np.array(Y_train)[sorted_idx[:k]].tolist()
            predict_class = max(set(top_k_class), key = top_k_class.count)
            predicts.append(predict_class)
        
        f1 = classification_report(Y_test, predicts, output_dict=True)['macro avg']['f1-score']
        f_scores.append(f1)

In [11]:
f_scores_split = np.array_split(f_scores, 6)

In [12]:
table = pd.DataFrame(f_scores_split, index=[1, 5, 10, 15, 20, 25], columns=['1_split', '2_split', '3_split', '4_split', '5_split'])

In [13]:
table['avg_score'] = table.sum(axis=1) / 5

In [14]:
table

,1_split,2_split,3_split,4_split,5_split,avg_score
1,0.944486,0.835920,0.829885,0.986364,0.891575,0.897646
5,0.750149,0.831818,0.762821,0.858238,0.747727,0.790150
10,0.814536,0.875630,0.795902,0.874505,0.762955,0.824706
15,0.808937,0.801533,0.762821,0.873167,0.762955,0.801882
20,0.808937,0.831818,0.779543,0.859848,0.762955,0.808620
25,0.775622,0.831818,0.745704,0.859848,0.747727,0.792144
